In [2]:
import sys
from pathlib import Path
import json
from pprint import pprint
from IPython.display import display, Markdown

# ensure project root (one level up from src) is on path if running notebook inside project root
PROJECT_ROOT = Path.cwd()
if str(PROJECT_ROOT / "src") not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT / "src"))

print("Project root:", PROJECT_ROOT)
print("sys.path[0]:", sys.path[0])

# Import your agent runner
from agent.agent_graph import run_agent 

# Optional: import retriever or tools for deeper debugging
from rag.retriever import retrieve_policy
from tools.tools import get_order_status, get_refund_status, get_inventory


Project root: c:\Coding\Agentic AI assignment
sys.path[0]: c:\Coding\Agentic AI assignment\src


In [ ]:
#Helper functions to pretty-print the agent State
from typing import Any, Dict

def brief_tool_resp(tool_resp: Dict[str, Any]):
    if not tool_resp:
        return None
    # show a compact summary of key fields when present
    for key in ("order_id","order_status","refund_status","in_stock","product_id","refund_amount"):
        if key in tool_resp:
            return {key: tool_resp.get(key)}
    # fallback: show top-level small keys
    keys = list(tool_resp.keys())[:6]
    out = {}
    for k in keys:
        v = tool_resp[k]
        if isinstance(v, (dict, list)):
            out[k] = repr(v)[:120] + ("..." if len(repr(v)) > 120 else "")
        else:
            s = str(v)
            out[k] = s if len(s) < 120 else s[:120] + "..."
    return out

def show_rag_snippets(snips):
    if not snips:
        display(Markdown("_No retrieved policy snippets._"))
        return
    for i, s in enumerate(snips, 1):
        meta = s.get("metadata", {}) or {}
        doc_id = meta.get("doc_id", "policy")
        combined = s.get("combined", None)
        header = f"**{i}. {doc_id}**   (score: {combined:.3f})" if combined is not None else f"**{i}. {doc_id}**"
        display(Markdown(header))
        txt = s.get("text", "").strip()
        display(Markdown("```text\n" + (txt[:2000] + ("..." if len(txt) > 2000 else "")) + "\n```"))
        display(Markdown("---"))

def display_state(state):
    display(Markdown("### Agent State Summary"))
    display(Markdown(f"- **User query:** {state.user_query}"))
    display(Markdown(f"- **Intent:** {state.intent}"))
    display(Markdown(f"- **Slots:** `{json.dumps(state.slots)}`"))
    display(Markdown("- **Tool response (brief):**"))
    tr = brief_tool_resp(state.tool_response)
    display(tr if tr else "_None_")
    display(Markdown("- **Top RAG snippets:**"))
    show_rag_snippets(state.rag_results)
    # Confidence: top snippet combined score if present
    confidence = 0.0
    if state.rag_results and isinstance(state.rag_results, list) and state.rag_results:
        top = state.rag_results[0]
        confidence = float(top.get("combined", 0.0))
    display(Markdown(f"- **Confidence (top snippet):** `{confidence:.3f}`"))
    display(Markdown("### Final Answer"))
    display(Markdown(state.final_answer or "_None_"))


In [4]:
# Cell 3: Run a single query interactively.
# Edit the variable 'QUERY' and run the cell.

QUERY = "Where is my order 98762?"   # <-- replace with any test query

state = run_agent(QUERY)
display_state(state)


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


### Agent State Summary

- **User query:** Where is my order 98762?

- **Intent:** order_status

- **Slots:** `{"order_id": "98762"}`

...

```text
DELIVERY DELAY POLICY

1. Standard Delivery Time
- Most orders are delivered within 2–5 business days.
- Remote or non-serviceable locations may take 5–10 business days.

2. Reasons for Delay
- High load at local distribution centers
- Weather disruptions
- Address verification issues
- Courier operational delays

3. Compensation Policy
- If the order is delayed beyond 3 days of the expected delivery date,
  customers may be eligible for delivery-fee refunds or discount coupons.

4. What Customers Should Do
- Check for address or contact number updates.
- Track order using “My Orders”.

5. When to Contact Support
- If order is delayed more than 5 days beyond expected date.
- If tracking stops updating for 72 hours.
```

---

- **Confidence (top snippet):** `0.850`

### Final Answer

Here’s what I found:

Order 98762 is currently: Out for delivery. Expected delivery: 2025-01-24. Reason: High load at local distribution center.

Relevant policy excerpts:
- RETURN POLICY

1. Return Window
- Most items can be returned within 10 days of delivery.
- Electronics and high-value items have a 7-day return window.
- Items marked “Non-returnable” cannot be returned unless damaged/defective.

2. Return Eligibility
A return is accepted only if:
- The item is in original condition.
- All accessories, tags, and packaging are intact.
- IMEI/serial numbers match ou...

- DELIVERY DELAY POLICY

1. Standard Delivery Time
- Most orders are delivered within 2–5 business days.
- Remote or non-serviceable locations may take 5–10 business days.

2. Reasons for Delay
- High load at local distribution centers
- Weather disruptions
- Address verification issues
- Courier operational delays

3. Compensation Policy
- If the order is delayed beyond 3 days of the expected deliv...



Policy reference (for your records):
return_policy.txt, delivery_delay_policy.txt

If you want me to do an action (refund/replace/cancel), tell me and I can open a request.

In [ ]:
# Cell 4: Input loop (useful for quick back-and-forth testing).
# Run this cell and type queries in the prompt. Press Ctrl+C in the notebook kernel to stop.

print("Interactive loop. Press Ctrl+C to exit.")
while True:
    q = input("\nEnter query (or 'exit' to stop): ").strip()
    if not q:
        continue
    if q.lower() in ("exit","quit"):
        break
    s = run_agent(q)
    display_state(s)


Interactive loop. Press Ctrl+C to exit.


### Agent State Summary

- **User query:** where is my order

- **Intent:** order_status

- **Slots:** `{}`

...

```text
REPLACEMENT & DAMAGE POLICY

1. Replacement Window
- Items eligible for replacement can be exchanged within 7 days of delivery.
- Electronics and high-value products allow replacement only for functional issues.

2. Damage/Defect Conditions
Replacement is allowed if:
- Item is received damaged.
- Product is defective or not functioning as advertised.
- Item differs significantly from the product description.

3. Documentation Required
- Clear photos of the item and packaging.
- For electronics, a short video showing the defect may be required.

4. Replacement Process
- Request replacement via My Orders → “Replace Item”.
- A new item is dispatched once pickup of the defective item is confirmed.

5. Refund Instead of Replacement
- If replacement stock is unavailable, refund will be issued as per refund policy.

6. Ineligible Cases
- Damage caused by customer misuse.
- Items missing accessories or original packaging.
```

---

- **Confidence (top snippet):** `1.000`

### Final Answer

Here’s what I found:

Relevant policy excerpts:
- REFUND POLICY

1. Refund Timelines
- Refunds are processed within 5–7 business days after the returned item clears QC.
- For cancellations before shipment, refunds are initiated immediately.

2. Payment Method Refund Times
- UPI / Wallet: 1–2 business days
- Credit/Debit Card: 3–5 business days
- Net Banking: 2–4 business days
- COD Orders: Refunded to bank account after verification (2–3 business...

- RETURN POLICY

1. Return Window
- Most items can be returned within 10 days of delivery.
- Electronics and high-value items have a 7-day return window.
- Items marked “Non-returnable” cannot be returned unless damaged/defective.

2. Return Eligibility
A return is accepted only if:
- The item is in original condition.
- All accessories, tags, and packaging are intact.
- IMEI/serial numbers match ou...

- REPLACEMENT & DAMAGE POLICY

1. Replacement Window
- Items eligible for replacement can be exchanged within 7 days of delivery.
- Electronics and high-value products allow replacement only for functional issues.

2. Damage/Defect Conditions
Replacement is allowed if:
- Item is received damaged.
- Product is defective or not functioning as advertised.
- Item differs significantly from the product d...



Policy reference (for your records):
refund_policy.txt, return_policy.txt, replacement_damage_policy.txt

If you want me to do an action (refund/replace/cancel), tell me and I can open a request.